In [1]:
from mitools import reg
from linearmodels.datasets import french
import seaborn as sns

# Linear Factor from Linearmodels

https://bashtage.github.io/linearmodels/asset-pricing/examples/examples.html

## Import data

The data used comes from Ken French”s [website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) and includes 4 factor returns, the excess market, the size factor, the value factor and the momentum factor. The available test portfolios include the 12 industry portfolios, a subset of the size-value two way sort, and a subset of the size-momentum two way sort.



In [ ]:
data = french.load()
print(french.DESCR)

## Transform the portfolios to be excesses

Subtract the risk-free rate from the test portfolios since these are not zero-investment.

In [3]:
data.iloc[:, 6:] = data.iloc[:, 6:].values - data[["RF"]].values

## 1-Step Estimation using Seemingly Unrelated Regression (SUR)

When the factors are traded assets, they must price themselves, and so the observed factor returns can be used to consistently estimate the expected factor returns. This also allows a set of seemingly unrelated regressions where each test portfolio is regressed on the factors and a constant to estimate factor loadings and $\alpha$ s.

Note that when using this type of model, it is essential that the test portfolios are excess returns. This is not a requirement of the other factor model estimators.

This specification is a CAP-M since only the market is included. The J-statistic tests whether all model $\alpha$ s are 0. The CAP-M is unsurprisingly unable to price the test portfolios.

In [ ]:
portfolios = ["S1V1", "S1V3", "S1V5", "S3V1", "S3V3", "S3V5", "S5V1", "S5V3", "S5V5"]
factors = ["MktRF"]
model = reg.TradedFactorModel(data=data, portfolios=portfolios, factors=factors)
res = model.fit()
print(res)


The factor set is expanded to include both the size and the value factors.

While the extra factors lower the J statistic and increases the $R^{2}$, the model is still easily rejected.

In [ ]:
factors = ["MktRF", "SMB", "HML"]
model = reg.TradedFactorModel(data=data, portfolios=portfolios, factors=factors)
res = model.fit()
print(res)

Changing the test portfolios to include only the industry portfolios does not allow factors to price the assets.

In [ ]:
indu = [
    "NoDur",
    "Durbl",
    "Manuf",
    "Enrgy",
    "Chems",
    "BusEq",
    "Telcm",
    "Utils",
    "Shops",
    "Hlth",
    "Money",
    "Other",
]
model = reg.TradedFactorModel(data=data, portfolios=indu, factors=factors)
res = model.fit()
print(res)

The estimated factor loadings ($\beta$ s) can be displayed using the `betas` property. There is reasonable dispersion in the factor loadings for all of the factors, except possibly the market which are all close to unity.

In [ ]:
print(res.betas)

In [ ]:
sns.heatmap(res.betas)

Similarly the $\alpha$ s can be displayed. These are monthly returns, and so scaling by 12 shows annualized returns. Healthcare has the largest pricing error.

In [ ]:
12 * res.alphas

## 2-Step Estimation

When one of more factors are not returns on traded assets, it is necessary to use a two-step procedure (or GMM). In the 2-step estimator, the first estimates factor loadings and the second uses the factor loadings to estimate the risk premia.

Here all four factors are used to attempt to price the size-momentum portfolios.

In [ ]:
factors = ["MktRF", "SMB", "HML", "Mom"]
portfolios = ["S1M1", "S1M3", "S1M5", "S3M1", "S3M3", "S3M5", "S5M1", "S5M3", "S5M5"]
model = reg.LinearFactorModel(data=data, portfolios=portfolios, factors=factors)
res = model.fit()
print(res)

In [ ]:
print(res.betas)

The identification requirements for this model are the $\beta$ s have unique variation – this requires some cross-sectional differences in exposures and that the loadings are not excessively correlated. Since these test portfolios do not attempt to sort on value, it is relatively non-dispersed and also correlated with both the market and size. This might make the inference unreliable.

In [ ]:
print(res.betas.corr())

The size factor was insignificant and so is dropped. This does not have much of an effect on the estimates.

In [ ]:
factors = ["MktRF", "HML", "Mom"]
portfolios = ["S1M1", "S1M3", "S1M5", "S3M1", "S3M3", "S3M5", "S5M1", "S5M3", "S5M5"]
model = reg.LinearFactorModel(data=data, portfolios=portfolios, factors=factors)
print(model.fit())

The risk-free rate can be optionally estimated. This is useful in case excess returns are not available of if the risk-free rate used to construct the excess returns might be misspecified.

Here the estimated risk-free rate is small and insignificant and has little impact on the model J-statistic.

In [ ]:
factors = ["MktRF", "HML", "Mom"]
portfolios = ["S1M1", "S1M3", "S1M5", "S3M1", "S3M3", "S3M5", "S5M1", "S5M3", "S5M5"]
model = reg.LinearFactorModel(
    data=data, portfolios=portfolios, factors=factors, risk_free=True
)
print(model.fit())

The default covariance estimator allows for heteroskedasticity but assumes there is no autocorrelation in the model errors or factor returns. Kernel-based HAC estimators (e.g. Newey-West) can be used by setting `cov_type="kernel"`.

This reduces the J-statistic indicating there might be some serial correlation although the model is still firmly rejected.

In [ ]:
model = reg.LinearFactorModel(data=data, portfolios=portfolios, factors=factors)
print(model.fit(cov_type="kernel"))

## GMM Estimation

The final estimator is the GMM estimator which is similar to estimating the 2-step model in a single step. In practice the GMM estimator is estimated at least twice, once to get an consistent estimate of the covariance of the moment conditions and the second time to efficiently estimate parameters.

The GMM estimator does not have a closed form and so a non-linear optimizer is required. The default output prints the progress every 10 iterations. Here the model is fit twice, which is the standard method to implement efficient GMM.

## GMM Estimation

The final estimator is the GMM estimator which is similar to estimating the 2-step model in a single step. In practice the GMM estimator is estimated at least twice, once to get an consistent estimate of the covariance of the moment conditions and the second time to efficiently estimate parameters.

The GMM estimator does not have a closed form and so a non-linear optimizer is required. The default output prints the progress every 10 iterations. Here the model is fit twice, which is the standard method to implement efficient GMM.

In [ ]:
model = reg.LinearFactorGMMModel(data=data, portfolios=portfolios, factors=factors)
res = model.fit()
print(res)

Kernel HAC estimators can be used with this estimator as well. Using a kernel HAC covariance also implies a Kernel HAC weighting matrix estimator.

Here the GMM estimator along with the HAC estimator indicates that these factors might be able to price this set of 9 test portfolios. `disp=0` is used to suppress iterative output.

In [ ]:
res = model.fit(cov_type="kernel", kernel="bartlett", disp=0)
print(res)

### Iterating until convergence

The standard approach is efficient and uses 2-steps. The first consistently estimates parameters using a sub-optimal weighting matrix, and the second uses the optimal weighting matrix conditional using the first stage estimates.

This method can be repeated until convergence, or for a fixed number of steps using the `steps` keyword argument.

In [ ]:
res = model.fit(steps=10, disp=25)
print(res)

## Continuosly Updating Estimator

The Continuously Updating Estimator (CUE) is optionally available using the flag `use_cue`. CUE jointly minimizes the J-statistic as a function of the moment conditions and the weighting matrix, rather than iterating between minimizing the J-statistic for a fixed weighting matrix and updating the weighting matrix.

Here the results are essentially the same as in the iterative approach.

In [ ]:
res = model.fit(use_cue=True)
print(res)

***